### nn.Module을 상속받아 Custom Model 생성하기
* 입력 Feature 갯수가 784이고 출력 feature 갯수가 100인 Linear Layer와 ReLU Activation Layer, 최종 10개의 출력 feature를 가지는 Linear Layer를 기반으로 모델 생성.
* \_\_init\_\_(self,..)에서 해당 Layer들 선언
* forward(self, x)에서 입력 tensor의 forward pass를 기술하면서 이들 Layer들을 연결
* 모델 입력은 반드시 tensor가 되어야 하며, 출력도 tensor가 됨. 

In [1]:
import torch
import torch.nn as nn
#from torch import nn

# Custom Model 생성. 
class LinearModel(nn.Module):
    def __init__(self, num_classes=10):
        # 반드시 super()를 호출. 
        super().__init__()
        #Linear Layer와 ReLU Layer 생성. 
        self.linear_01 = nn.Linear(in_features=784, out_features=100)
        self.relu_01 = nn.ReLU()
        self.linear_02 = nn.Linear(in_features=100, out_features=num_classes)
        
    # 순방향 전파(Pass Forward) 기술.
    def forward(self, x):
        x = self.linear_01(x)
        x = self.relu_01(x)
        output = self.linear_02(x)
        return output

In [2]:
#임의의 입력 tensor 생성. 
input_tensor = torch.randn(size=(64, 784))
print(input_tensor.size())

# LinearModel 객체 생성. __init__(self, num_classes)에 선언된 객체 초기화 인자 입력하여 생성. 
linear_model = LinearModel(num_classes=10)

# LinearModel 객체는 Callable Object이므로 LinearModel 객체에 함수 호출과 유사한 형태로 입력 인자 전달하여 forward()메소드 호출. 
output_tensor = linear_model(input_tensor)
print(output_tensor.size())

torch.Size([64, 784])
torch.Size([64, 10])


### Layer(nn.Linear, nn.Conv2d, nn.ReLU, nn.MaxPool2d등) 살펴보기
* Pytorch의 Layer는 신경망(Neural Network)을 쉽게 생성하기 위한 직관적인 building block
* Layer 역시 nn.Module을 상속 받아 생성되며, 자동미분(Auto differentiation)과 GPU device 지원
* Layer들은 입력 데이터에 적용되는 구조와 변환을 기술하여 선형 변환(Liner transformation), Convolution 적용, 활성함수 적용(Activation), Pooling과 Normalization 등의 작업을 수행.
* 내부적으로 학습 파라미터를 가지고 있는 Layer(nn.Linear, nn.Conv2d)와 주로 변환만을 수행하는 Layer(nn.ReLU, nn.MaxPool2d)등이 있음.
* Callable Object의 생성과 입력 데이터 전달 방식과 비슷하게, 생성 인자를 입력하여 Layer객체를 생성 한 뒤 변환될 입력 tensor를 객체의 인자로 입력해 주는 방식으로 변환

In [3]:
import torch.nn as nn

linear_01= nn.Linear(in_features=784, out_features=100)
# 학습 파라미터를 가지고 있음.
print(linear_01.weight)
print(linear_01.bias)

# 학습 파라미터는 nn.parameter.Parameter 타입이며, nn.parameter.Parameter는 학습이 가능한(자동 미분) 특별한 타입의 Tensor임. 
print(type(linear_01.weight))

Parameter containing:
tensor([[-0.0056,  0.0183,  0.0219,  ..., -0.0115,  0.0077,  0.0054],
        [ 0.0126,  0.0324, -0.0273,  ...,  0.0109,  0.0268,  0.0040],
        [-0.0054, -0.0152, -0.0300,  ...,  0.0326, -0.0179, -0.0236],
        ...,
        [-0.0273, -0.0251, -0.0287,  ...,  0.0023, -0.0080,  0.0236],
        [-0.0081,  0.0039,  0.0295,  ..., -0.0174, -0.0330,  0.0265],
        [ 0.0168, -0.0001, -0.0025,  ..., -0.0008, -0.0114,  0.0149]],
       requires_grad=True)
Parameter containing:
tensor([ 9.8777e-03, -1.8693e-02,  3.1205e-02,  1.6275e-02, -2.7545e-03,
        -2.2487e-02, -1.6596e-02, -2.0752e-02, -1.7975e-02, -2.9267e-02,
         6.8150e-03,  2.1224e-02, -2.3022e-02, -1.1930e-02, -3.2589e-02,
         1.3139e-02,  7.4696e-03,  1.6216e-02, -2.1187e-02, -8.4596e-03,
        -1.2772e-02,  5.5135e-04, -2.5150e-02,  2.7795e-02, -1.6036e-02,
         1.3935e-02, -1.8359e-02,  3.1903e-02, -1.2559e-02,  3.9326e-05,
         2.1647e-03,  3.2502e-02, -1.7871e-02,  9.5639e-0

In [4]:
# linear_01= nn.Linear(in_features=784, out_features=100)
# weight의 shape는 matmul()을 위해서 (in_features, out_features)의 행렬 위치가 바뀌는 Transpose 적용되어야 함. 
print(linear_01.weight.shape, linear_01.bias.shape)

torch.Size([100, 784]) torch.Size([100])


#### nn.Parameter 는 학습이 가능한(자동 미분) 특별한 타입의 Tensor임. 
* nn.Module을 상속받은 모든 객체는 자신이 가지는 Parameter Tensor를 Optimizer에 등록할 수 있음.
* 일반 Tensor역시 requires_grad를 수행하면 자동 미분은 가능하지만 Optimizer에 등록 할 수는 없으므로 optimizer에서 grad upgrade를 수행할 수 없음.

In [5]:
# Layer의 parameters() 메소드는 Layer가 가지는 모든 parameter들을 iteration으로 반환함. 
for parameter in linear_01.parameters():
    print(parameter)

Parameter containing:
tensor([[-0.0056,  0.0183,  0.0219,  ..., -0.0115,  0.0077,  0.0054],
        [ 0.0126,  0.0324, -0.0273,  ...,  0.0109,  0.0268,  0.0040],
        [-0.0054, -0.0152, -0.0300,  ...,  0.0326, -0.0179, -0.0236],
        ...,
        [-0.0273, -0.0251, -0.0287,  ...,  0.0023, -0.0080,  0.0236],
        [-0.0081,  0.0039,  0.0295,  ..., -0.0174, -0.0330,  0.0265],
        [ 0.0168, -0.0001, -0.0025,  ..., -0.0008, -0.0114,  0.0149]],
       requires_grad=True)
Parameter containing:
tensor([ 9.8777e-03, -1.8693e-02,  3.1205e-02,  1.6275e-02, -2.7545e-03,
        -2.2487e-02, -1.6596e-02, -2.0752e-02, -1.7975e-02, -2.9267e-02,
         6.8150e-03,  2.1224e-02, -2.3022e-02, -1.1930e-02, -3.2589e-02,
         1.3139e-02,  7.4696e-03,  1.6216e-02, -2.1187e-02, -8.4596e-03,
        -1.2772e-02,  5.5135e-04, -2.5150e-02,  2.7795e-02, -1.6036e-02,
         1.3935e-02, -1.8359e-02,  3.1903e-02, -1.2559e-02,  3.9326e-05,
         2.1647e-03,  3.2502e-02, -1.7871e-02,  9.5639e-0

In [6]:
import torch

tensor_01 = torch.rand(size=(100, 784))
print(tensor_01.requires_grad)

param_01 = nn.Parameter(data=tensor_01)
print(param_01.shape, param_01.requires_grad)

False
torch.Size([100, 784]) True


### 모델 구성 모듈(Layer 및 서브모듈) 살펴보기
* 서브모듈(submodule)은 nn.Module을 상속 받아 생성된 컴포넌트(Layer도 서브모듈). 보통은 여러개의 Layer등을 엮어서 만들어지는 또 다른 클래스(블록)라는 의미로 통용되며, 모델이 복잡한 구조로 되어 있을 경우 특정 구조를 작게 블록화 시키는데 적용됨. 
* 서브모듈 또한 nn.Module에서 제공하는 여러기능을 가지게 됨(자식 모듈 등록, 파라미터 자동 등록 및 자동 미분 수행등)
* 서브모듈 생성 시 반드시 생성자 메소드와  forward() 메소드를 구현해야 함.
* 강의 진행 시 용어 정리는 아래와 같이 하겠음.
    * 모델: 최종으로 만들어지는 네트웍 모델
    * 서브모듈(또는 Block): 여러개의 Layer로 연결되어 만들어 지는 블록형 모듈.
    * 서브모듈(모듈): Layer와 서브모듈등 nn.Module을 상속받은 모든 객체
    * Layer: nn.Linear와 같은 Layer
    * nn.Module: nn.Module

In [7]:
import torch
import torch.nn as nn

# 서브 모듈 생성.
class SimpleBlock(nn.Module):
    def __init__(self, in_features, out_features):
        super().__init__()
        self.linear_01 = nn.Linear(in_features=in_features,
                                   out_features=out_features)
        self.relu_01 = nn.ReLU()

    def forward(self, x):
        x = self.linear_01(x)
        x = self.relu_01(x)
        return x

class LinearModel(nn.Module):
    def __init__(self, num_classes=10):
        # 반드시 super()를 호출. 
        super().__init__()
        # 서브 모듈인 SimpleBlock 생성
        self.simple_01 = SimpleBlock(in_features=784,
                                     out_features=100)
        self.linear_02 = nn.Linear(in_features=100, out_features=num_classes)
        
    # 순방향 전파(Pass Forward) 기술.
    def forward(self, x):
        x = self.simple_01(x)
        output = self.linear_02(x)
        return output

In [8]:
input_tensor = torch.randn(size=(64, 784))
print(input_tensor.size())

# LinearModel 객체 생성. __init__(self, num_classes)에 선언된 객체 초기화 인자 입력하여 생성. 
linear_model = LinearModel(num_classes=10)

# LinearModel 객체는 Callable Object이므로 LinearModel 객체에 함수 호출과 유사한 형태로 입력 인자 전달하여 forward()메소드 호출. 
output_tensor = linear_model(input_tensor)
print(output_tensor.size())

torch.Size([64, 784])
torch.Size([64, 10])


#### 모델이 가지는 모든 모듈(Layer, 서브모듈) 확인하기
* modules() 메소드는 자신의 모듈을 포함하여 Nesting된 서브 모듈(Layer포함하여 nn.Module을 상속받은 모든 클래스)를 출력
* named_modules() 메소드는 자신의 모듈을 포함하여 Nesting된 서브 모듈까지 모듈명과 모듈 클래스를 출력
* 모델이 가지는 내부 멤버변수(self로 지정된 변수)는 객체명.변수명으로 바로 접근 가능

In [9]:
# 모델 구성 출력
print(linear_model)

LinearModel(
  (simple_01): SimpleBlock(
    (linear_01): Linear(in_features=784, out_features=100, bias=True)
    (relu_01): ReLU()
  )
  (linear_02): Linear(in_features=100, out_features=10, bias=True)
)


In [10]:
# 자신의 모듈을 포함하여 Nesting된 서브 모듈까지 모두 출력
for module in linear_model.modules():
    print(module)

LinearModel(
  (simple_01): SimpleBlock(
    (linear_01): Linear(in_features=784, out_features=100, bias=True)
    (relu_01): ReLU()
  )
  (linear_02): Linear(in_features=100, out_features=10, bias=True)
)
SimpleBlock(
  (linear_01): Linear(in_features=784, out_features=100, bias=True)
  (relu_01): ReLU()
)
Linear(in_features=784, out_features=100, bias=True)
ReLU()
Linear(in_features=100, out_features=10, bias=True)


In [11]:
# named_modules() 메소드는 자신의 모듈을 포함하여 Nesting된 서브 모듈까지 모듈명과 모듈 클래스를 출력
for name, module in linear_model.named_modules():
    print(f"Module Name: {name}, Module: {module}")

Module Name: , Module: LinearModel(
  (simple_01): SimpleBlock(
    (linear_01): Linear(in_features=784, out_features=100, bias=True)
    (relu_01): ReLU()
  )
  (linear_02): Linear(in_features=100, out_features=10, bias=True)
)
Module Name: simple_01, Module: SimpleBlock(
  (linear_01): Linear(in_features=784, out_features=100, bias=True)
  (relu_01): ReLU()
)
Module Name: simple_01.linear_01, Module: Linear(in_features=784, out_features=100, bias=True)
Module Name: simple_01.relu_01, Module: ReLU()
Module Name: linear_02, Module: Linear(in_features=100, out_features=10, bias=True)


In [12]:
# named_children() 메소드는 자기 직계 서브 모듈만 모듈명과 모듈 클래스 출력
for name, module in linear_model.named_children():
    print(f"Submodule Name: {name}, Submodule: {module}")

Submodule Name: simple_01, Submodule: SimpleBlock(
  (linear_01): Linear(in_features=784, out_features=100, bias=True)
  (relu_01): ReLU()
)
Submodule Name: linear_02, Submodule: Linear(in_features=100, out_features=10, bias=True)


In [13]:
#모델이 가지는 내부 멤버변수(self로 지정된 변수)는 객체명.변수명으로 바로 접근 가능
print('simple_01:', linear_model.simple_01)
print('linear_02:', linear_model.linear_02)
print('linear_01 in simple_01:', linear_model.simple_01.linear_01)

simple_01: SimpleBlock(
  (linear_01): Linear(in_features=784, out_features=100, bias=True)
  (relu_01): ReLU()
)
linear_02: Linear(in_features=100, out_features=10, bias=True)
linear_01 in simple_01: Linear(in_features=784, out_features=100, bias=True)


#### 모델의 모든 Parameter 가져오기
* nn.Module을 상속 받은 모든 클래스는 등록된 Parameter Tensor를 parameters()로 가져 올 수 있음.
* 모델이 가지는 서브모듈들의 모든 parameter들을 parameters()로 가져 올 수 있음.
* named_parameter()는 parameter를 가지는 서브모듈/Layer의 weight/bias와 parameter tensor를 모두 출력 

In [14]:
for parameter in linear_model.parameters():
    print(parameter)
# for name, parameter in linear_model.named_parameters():
#     print(name, parameter)

Parameter containing:
tensor([[-0.0048,  0.0105,  0.0313,  ..., -0.0306, -0.0232, -0.0229],
        [ 0.0094,  0.0275,  0.0215,  ..., -0.0352,  0.0215, -0.0234],
        [-0.0076,  0.0288, -0.0229,  ..., -0.0080,  0.0225, -0.0232],
        ...,
        [ 0.0232,  0.0188, -0.0173,  ..., -0.0006,  0.0336, -0.0308],
        [-0.0108, -0.0002,  0.0268,  ..., -0.0135,  0.0225,  0.0349],
        [ 0.0219,  0.0156, -0.0337,  ..., -0.0017,  0.0205, -0.0292]],
       requires_grad=True)
Parameter containing:
tensor([ 2.3991e-02,  5.2879e-03,  3.7704e-04, -3.2616e-02, -3.6454e-03,
         1.0816e-02,  5.7917e-03, -1.1305e-03, -2.3250e-02,  3.9217e-03,
         1.9031e-02,  8.8242e-03, -3.2714e-02, -2.0103e-02,  3.2724e-02,
        -3.3922e-02, -2.5260e-02,  2.6187e-02, -1.7119e-03,  1.1702e-02,
         2.3196e-02, -3.2545e-02,  7.6600e-03,  9.0178e-04,  2.4319e-02,
         3.1890e-02, -3.4113e-03,  2.1267e-02, -4.4083e-03,  3.0923e-02,
         1.2813e-03,  2.8033e-02,  3.0571e-02, -2.9288e-0

### torchinfo의 summary()
* torchinfo 패키지의 summary()를 이용하여 보다 자세히 모델 구조를 확인 할 수 있음.
    * model: 모델 객체
    * input_size: 입력 tensor 사이즈. 일반적으로 batch 를 감안하여 입력. 
    * col_names: summary 수행 출력 컬럼명들. list형태로 입력.
        * input_size: 입력 tensor size
        * output_size: 출력 tensor size
        * num_params: 학습 파라미터 갯수
        * trainable: 학습 파라미터의 train 가능 설정(requires_grad) 
    * row_settings: row에 보여질 내용
        * var_names: 모듈 변수명
        * depth: 서브 모듈내 depth

In [ ]:
#!pip install torchinfo

In [15]:
input_tensor = torch.randn(size=(64, 784))
print(input_tensor.size())

# LinearModel 객체 생성. __init__(self, num_classes)에 선언된 객체 초기화 인자 입력하여 생성. 
linear_model = LinearModel(num_classes=10)

# LinearModel 객체는 Callable Object이므로 LinearModel 객체에 함수 호출과 유사한 형태로 입력 인자 전달하여 forward()메소드 호출. 
output_tensor = linear_model(input_tensor)
print(output_tensor.size())

torch.Size([64, 784])
torch.Size([64, 10])


In [16]:
from torchinfo import summary

summary(model=linear_model, input_size=(64, 784),
        col_names=['input_size', 'output_size', 'num_params'], #'trainable'
        row_settings=['var_names', 'depth'],
        depth=3
       )

Layer (type (var_name):depth-idx)        Input Shape               Output Shape              Param #
LinearModel (LinearModel)                [64, 784]                 [64, 10]                  --
├─SimpleBlock (simple_01): 1-1           [64, 784]                 [64, 100]                 --
│    └─Linear (linear_01): 2-1           [64, 784]                 [64, 100]                 78,500
│    └─ReLU (relu_01): 2-2               [64, 100]                 [64, 100]                 --
├─Linear (linear_02): 1-2                [64, 100]                 [64, 10]                  1,010
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
Total mult-adds (Units.MEGABYTES): 5.09
Input size (MB): 0.20
Forward/backward pass size (MB): 0.06
Params size (MB): 0.32
Estimated Total Size (MB): 0.58